In [340]:
#data load, concat(ref=SEQN), dropna

import pandas as pd
import numpy as np
from functools import reduce

#data load, feature name alteration
df1 = pd.read_sas(r"C:\Users\cmc\Desktop\monash AI\data\SMQ_L.xpt", format='xport')
df_fil1=df1[['SEQN','SMQ040']]

df2 = pd.read_sas(r"C:\Users\cmc\Desktop\monash AI\data\PAQ_L.xpt", format='xport')
df_fil2=df2[['SEQN','PAD790Q','PAD790U','PAD800','PAD810Q','PAD810U','PAD820']] 

df3 = pd.read_sas(r"C:\Users\cmc\Desktop\monash AI\data\ALQ_L.xpt", format='xport')
df_fil3=df3[['SEQN','ALQ121','ALQ130']] 

#data concat
dfs = [ df_fil1, df_fil2, df_fil3]  #smk는 결측치 많아서 제거함
df_merged = reduce(
    lambda left, right: pd.merge(left, right, on="SEQN", how="inner"),
    dfs
)

#data dropna

#column rename
df_merged2 = df_merged.rename(columns={
    'SEQN': 'subject_no',
    'SMQ040': 'smk',
    'PAD790Q': 'mpa_f',
    'PAD790U':'mpa_u',
    'PAD800':'mpa_t',
    'PAD810Q':'vpa_f',
    'PAD810U':'vpa_u',
    'PAD820':'vpa_t',
    'ALQ121':'ac_f',
    'ALQ130':'ac_t'
})

df_merged2.shape

(6337, 10)

In [341]:
df_merged2.isna().sum()

subject_no       0
smk           3819
mpa_f            7
mpa_u            0
mpa_t         1306
vpa_f            7
vpa_u            0
vpa_t         3460
ac_f          1415
ac_t          2268
dtype: int64

In [342]:
df_merged2=df_merged2.dropna(subset=['mpa_f','mpa_u','vpa_f','vpa_u','mpa_t','vpa_t','ac_f','ac_t'])
df_merged2.shape

(1994, 10)

In [343]:
df_merged2
map_dict = {b'D': 0, b'M': 1, b'W': 2, b'Y': 3}

df_merged2['mpa_u'] = df_merged2['mpa_u'].map(map_dict)
df_merged2['vpa_u'] = df_merged2['vpa_u'].map(map_dict)


In [344]:
df_merged2 = df_merged2[df_merged2['mpa_t'] <= 300.0]
df_merged2[['mpa_t']].value_counts()  
counts = df_merged2['mpa_t'].value_counts()
counts_sorted = counts.sort_index()
#print(counts_sorted)

df_merged2 = df_merged2[df_merged2['vpa_t'] <= 300.0]
df_merged2[['vpa_t']].value_counts()  
counts = df_merged2['vpa_t'].value_counts()
counts_sorted = counts.sort_index()
#print(counts_sorted)

df_merged2 = df_merged2[df_merged2['ac_t'] <= 100.0]
df_merged2[['ac_t']].value_counts()  
counts = df_merged2['ac_t'].value_counts()
counts_sorted = counts.sort_index()
print(counts_sorted)

ac_t
1.0     617
2.0     658
3.0     315
4.0     126
5.0      93
6.0      66
7.0       7
8.0      17
9.0       7
10.0     17
12.0     21
13.0      1
14.0      2
15.0     10
Name: count, dtype: int64


In [345]:
def to_week(freq, unit):
    """
    freq: frequency
    unit: 1=day, 2=week, 3=month, 4=year
    """

    if unit == 0:    # per day
        return freq * 7
    elif unit == 1:  # per month
        return freq / 4.345
    elif unit == 2:  # per week
        return freq 
    elif unit == 3:  # per year
        return freq / 52
    else:
        return np.nan

# --- 중강도(mpa) 주당 총 시간 ---
df_merged2['mpa_per_week'] = df_merged2.apply(
    lambda r: to_week(r['mpa_f'], r['mpa_u']) * r['mpa_t']
    if not pd.isna(r['mpa_t']) else np.nan,
    axis=1
)

# --- 고강도(vpa) 주당 총 시간 ---
df_merged2['vpa_per_week'] = df_merged2.apply(
    lambda r: to_week(r['vpa_f'], r['vpa_u']) * r['vpa_t']
    if not pd.isna(r['vpa_t']) else np.nan,
    axis=1
)

# --- MVPA 계산 (고강도는 2배 가중) ---
df_merged2['mvpa'] = df_merged2['mpa_per_week'] + 2 * df_merged2['vpa_per_week']
df_merged3=df_merged2[['subject_no','mvpa','ac_f','ac_t']]


In [346]:
df_merged3

,subject_no,mvpa,ac_f,ac_t
1,130379.0,450.000000,2.0,3.0
3,130386.0,43.808976,4.0,2.0
6,130389.0,162.617952,2.0,2.0
10,130393.0,300.000000,3.0,2.0
11,130394.0,555.000000,6.0,1.0
...,...,...,...,...
6318,142284.0,780.000000,5.0,2.0
6322,142289.0,540.000000,10.0,1.0
6323,142290.0,660.000000,3.0,6.0
6327,142299.0,840.000000,9.0,2.0


In [347]:
import numpy as np
import pandas as pd

# ac_f(빈도) → 연간 횟수 매핑
ac_f_to_year = {
    1: 365,
    2: 300,
    3: 182,
    4: 104,
    5: 52,
    6: 30,
    7: 12,
    8: 9,
    9: 4.5,
    10: 1.5,
    0: 0,      # Never in the last year
    836: 0,    # Never (NHANES recoded)
    77: np.nan,
    99: np.nan
}

# 연간 → 주당 빈도
df_merged3["ac_freq_per_year"] = df_merged3["ac_f"].map(ac_f_to_year)
df_merged3["ac_freq_per_week"] = df_merged3["ac_freq_per_year"] / 52

# 알코올 주당 잔수 계산
df_merged3["alcohol_drinks_per_week"] = df_merged3["ac_freq_per_week"] * df_merged3["ac_t"]



C:\Users\cmc\AppData\Local\Temp\ipykernel_31892\1471328081.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged3["ac_freq_per_year"] = df_merged3["ac_f"].map(ac_f_to_year)
C:\Users\cmc\AppData\Local\Temp\ipykernel_31892\1471328081.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged3["ac_freq_per_week"] = df_merged3["ac_freq_per_year"] / 52
C:\Users\cmc\AppData\Local\Temp\ipykernel_31892\1471328081.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [348]:
df_merged3 = df_merged3.rename(columns={
    'alcohol_drinks_per_week': 'ac_week',
})
df_merged3

,subject_no,mvpa,ac_f,ac_t,ac_freq_per_year,ac_freq_per_week,ac_week
1,130379.0,450.000000,2.0,3.0,300.0,5.769231,17.307692
3,130386.0,43.808976,4.0,2.0,104.0,2.000000,4.000000
6,130389.0,162.617952,2.0,2.0,300.0,5.769231,11.538462
10,130393.0,300.000000,3.0,2.0,182.0,3.500000,7.000000
11,130394.0,555.000000,6.0,1.0,30.0,0.576923,0.576923
...,...,...,...,...,...,...,...
6318,142284.0,780.000000,5.0,2.0,52.0,1.000000,2.000000
6322,142289.0,540.000000,10.0,1.0,1.5,0.028846,0.028846
6323,142290.0,660.000000,3.0,6.0,182.0,3.500000,21.000000
6327,142299.0,840.000000,9.0,2.0,4.5,0.086538,0.173077


In [349]:
df_merged4=df_merged3[['subject_no','mvpa','ac_week']]
df_merged4

,subject_no,mvpa,ac_week
1,130379.0,450.000000,17.307692
3,130386.0,43.808976,4.000000
6,130389.0,162.617952,11.538462
10,130393.0,300.000000,7.000000
11,130394.0,555.000000,0.576923
...,...,...,...
6318,142284.0,780.000000,2.000000
6322,142289.0,540.000000,0.028846
6323,142290.0,660.000000,21.000000
6327,142299.0,840.000000,0.173077


In [350]:
df_merged4 = df_merged4[df_merged4['mvpa'] <= 2000.0]
df_merged4[['mvpa']].value_counts()  
counts = df_merged4['mvpa'].value_counts()
counts_sorted = counts.sort_index()
print(counts_sorted)

mvpa
1.826923       1
2.884615       1
4.423077       1
5.480769       1
8.058334       1
              ..
1790.471807    1
1800.000000    9
1890.000000    1
1920.000000    4
1980.000000    3
Name: count, Length: 471, dtype: int64


In [351]:
df_merged4 = df_merged4[df_merged4['ac_week'] <= 60.0]
df_merged4[['ac_week']].value_counts()  
counts = df_merged4['ac_week'].value_counts()
counts_sorted = counts.sort_index()
print(counts_sorted)

ac_week
0.028846     118
0.057692      32
0.086538      97
0.115385       5
0.144231       2
            ... 
49.134615      1
51.923077      1
52.500000      2
56.153846      2
57.692308      1
Name: count, Length: 85, dtype: int64


In [352]:
df_merged4

,subject_no,mvpa,ac_week
1,130379.0,450.000000,17.307692
3,130386.0,43.808976,4.000000
6,130389.0,162.617952,11.538462
10,130393.0,300.000000,7.000000
11,130394.0,555.000000,0.576923
...,...,...,...
6318,142284.0,780.000000,2.000000
6322,142289.0,540.000000,0.028846
6323,142290.0,660.000000,21.000000
6327,142299.0,840.000000,0.173077


In [353]:
df_merged4.to_pickle("df_ques.pkl")
